In [2]:
#############################################################################
#
# Make SeedEating Movies
# 08/31/19
#
#############################################################################
import numpy as np
import os, sys
from datetime import datetime as dt
import importlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import json
import random
import re
from PIL import Image, ImageFont, ImageDraw
import cv2
import dlib
import argparse
import imutils
import time
from imutils.video import VideoStream
import statistics
from pathlib import Path
import csv
from scipy.stats import t
from holdbite_utils import *
from cv_utils import *

In [3]:
#############################################################################
#
# Define data folders
#
#############################################################################

dDrive = "D:%s" % os.sep
eatingBoxFolder = 'eating_box'
videoFolder = 'FullVideos'
dayFolder = '20190801-full'

dayPath = os.path.join(dDrive, eatingBoxFolder, videoFolder, dayFolder)

In [ ]:
myFile = os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam1-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv')

In [5]:
#############################################################################
#
# Takes list of paths to videos and creates synced video
#
# input arguments:
# imgFolderList - (list of strings) full paths to 2 or 3 videos
# saveTo        - (string) full path to folder where movie will be saved
# movieName     - (string) name of video file to be saved
# 
#############################################################################
def syncLabeledVideos(vidFileList,labelFileList,saveTo, movieName, h=480,w=640,codec='XVID',ext='.avi',fontSize=40):
    #Initialize variables
    vidFile1 = vidFileList[0]
    cap1 = cv2.VideoCapture(vidFile1)
    bodyParts1 = loadLabels(labelFileList[0])
    
    vidFile2 = vidFileList[1]
    cap2 = cv2.VideoCapture(vidFile2)
    bodyParts2 = loadLabels(labelFileList[1])
    
    try:
        vidFile3 = vidFileList[2]
        cap3 = cv2.VideoCapture(vidFile3)
        bodyParts3 = loadLabels(labelFileList[2])
        hasThirdVideo = True
    except:
        hasThirdVideo = False
        print('No third video.')
    movieName = movieName + ext
   
    fps = cap1.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*codec)
    writer = cv2.VideoWriter(os.path.join(saveTo, movieName), fourcc, fps, (2*w, 2*h), True)

    #Main for loop
    #Opens image, adds text to image, adds image to movie writer
    print('Frame Rate:', fps)
    
        
    font = ImageFont.truetype(os.path.join("D:%s" % os.sep, 'python_code', 'Roboto', 'Roboto-Black.ttf'), fontSize)
    
    #draw object for info panel
    infoImg = Image.new('RGB', (w, h))
    drawObjTL = ImageDraw.Draw(infoImg)
    xpos = 30
    ypos = 10
    yinc = fontSize
    text_color = (255, 255, 255)
    colorList = [(180, 230, 245),
                (0, 71, 160),
                (36, 211, 154),
                (205, 2, 43)]
    bodyPartCols = {}
    for p, part in enumerate(bodyParts1.keys()):
        bodyPartCols.update({part:colorList[p]})
    

    for part in bodyPartCols:
        drawObjTL.text((xpos, ypos), part, fill=bodyPartCols[part], font=font)
        drawObjTL.ellipse([5, ypos+15, 25, ypos+35], fill=bodyPartCols[part])
        ypos += yinc
        
    ypos += yinc
    lcutoff = 0.1
    drawObjTL.text((5, ypos), 'likelihood>%.2f' %lcutoff, fill=text_color, font=font)
        
    topLeftPanel = np.flip(np.array(infoImg), axis=2)
    if not hasThirdVideo:
        bottomLeftPanel = np.zeros((h,w))
        
    markerRadius = 5
    
    ret = True
    frameNum = 0
    while ret:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if hasThirdVideo:
            ret3, frame3 = cap3.read()
            ret = ret1 and ret2 and ret3
        else:
            ret = ret1 and ret2
        if not ret:
            break
            
        topRightPanelImg = Image.fromarray(frame1).convert('RGB')
        drawObjTR = ImageDraw.Draw(topRightPanelImg)
        for part in bodyParts1:
            x = bodyParts1[part]['x'][frameNum]
            y = bodyParts1[part]['y'][frameNum]
            l = bodyParts1[part]['likelihood'][frameNum]
            ladd = np.log10(l)
            xy = [int(x-(markerRadius+ladd)), int(y-(markerRadius+ladd)), 
                  int(x+markerRadius+ladd), int(y+(markerRadius+ladd))]
            if l>lcutoff:
                drawObjTR.ellipse(xy,
                              fill=bodyPartCols[part], outline=None)
        topRightPanel = np.flip(np.array(topRightPanelImg), axis=2)
        
        bottomRightPanelImg =Image.fromarray(frame2).convert('RGB')
        drawObjBR = ImageDraw.Draw(bottomRightPanelImg)
        for part in bodyParts2:
            x = bodyParts2[part]['x'][frameNum]
            y = bodyParts2[part]['y'][frameNum]
            l = bodyParts2[part]['likelihood'][frameNum]
            ladd = np.log10(l)
            xy = [int(x-(markerRadius+ladd)), int(y-(markerRadius+ladd)), 
                  int(x+markerRadius+ladd), int(y+(markerRadius+ladd))]
            if l>lcutoff:
                drawObjBR.ellipse(xy,
                              fill=bodyPartCols[part], outline=None)
        bottomRightPanel = np.flip(np.array(bottomRightPanelImg), axis=2)
        
        #look for closest frame time for bottom left panel
        if hasThirdVideo:
            bottomLeftPanelImg =Image.fromarray(frame3).convert('RGB')
            drawObjBL = ImageDraw.Draw(bottomLeftPanelImg)
            for part in bodyParts3:
                x = bodyParts3[part]['x'][frameNum]
                y = bodyParts3[part]['y'][frameNum]
                l = bodyParts3[part]['likelihood'][frameNum]
                ladd = np.log10(l)
                xy = [int(x-(markerRadius+ladd)), int(y-(markerRadius+ladd)), int(x+markerRadius+ladd), int(y+(markerRadius+ladd))]
                if l>lcutoff:
                    drawObjBL.ellipse(xy,fill=bodyPartCols[part], outline=None)
            bottomLeftPanel = np.flip(np.array(bottomLeftPanelImg), axis=2)
            
        topCat = np.hstack([topLeftPanel, topRightPanel])
        botCat = np.hstack([bottomLeftPanel, bottomRightPanel])
                                           
        outputFrame = np.vstack([topCat, botCat])

        writer.write(outputFrame)
        # show the frames
        cv2.imshow('outputFrame', outputFrame)
        frameNum+=1
        key = cv2.waitKey(1) & 0xFF
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # do a bit of cleanup
    print("[INFO] cleaning up...")
    cv2.destroyAllWindows()
    writer.release()

In [157]:
vidFileList = [os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam1-full.avi'),
              os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam2-full.avi'),
              os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam3-full.avi')]
labelFileList = [os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam1-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam2-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dayPath, '20190801-wt0-day04-fullseed-video-02-cam3-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv')]
syncLabeledVideos(vidFileList, labelFileList, dayPath, '20190801-wt0-day04-fullseed-video-02-synced')

d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Frame Rate: 30.0
[INFO] cleaning up...


In [165]:
dlcFolder = os.path.join(dDrive, 'DeepLabCut')
vidFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190801-wt0-day04-fullseed-video-03-cam1-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190801-wt0-day04-fullseed-video-03-cam2-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190801-wt0-day04-fullseed-video-03-cam3-full.avi')]
labelFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190801-wt0-day04-fullseed-video-03-cam1-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190801-wt0-day04-fullseed-video-03-cam2-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190801-wt0-day04-fullseed-video-03-cam3-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv')]
syncLabeledVideos(vidFileList, labelFileList, os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos'), '20190801-wt0-day04-fullseed-video-03-synced')

d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Frame Rate: 30.0
[INFO] cleaning up...


In [167]:
dlcFolder = os.path.join(dDrive, 'DeepLabCut')
vidFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x5-day02-crackedseed-video-02-cam1-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x5-day02-crackedseed-video-02-cam2-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x5-day02-crackedseed-video-02-cam3-full.avi')]
labelFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x5-day02-crackedseed-video-02-cam1-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x5-day02-crackedseed-video-02-cam2-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x5-day02-crackedseed-video-02-cam3-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv')]
syncLabeledVideos(vidFileList, labelFileList, os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos'), '20190806-x5-day02-crackedseed-video-02-synced')

d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Frame Rate: 30.0
[INFO] cleaning up...


In [169]:
dlcFolder = os.path.join(dDrive, 'DeepLabCut')
vidFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-02-cam1-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-02-cam2-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-02-cam3-full.avi')]
labelFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-02-cam1-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-02-cam2-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-02-cam3-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv')]
syncLabeledVideos(vidFileList, labelFileList, os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos'), '20190806-x4-day02-crackedseed-video-02-synced')

d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Frame Rate: 30.0
[INFO] cleaning up...


In [6]:
dlcFolder = os.path.join(dDrive, 'DeepLabCut')
vidFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-03-cam1-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-03-cam2-full.avi'),
              os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-03-cam3-full.avi')]
labelFileList = [os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-03-cam1-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-03-cam2-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv'),
                os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos', '20190806-x4-day02-crackedseed-video-03-cam3-fullDeepCut_resnet50_SeedEatingAug26shuffle2_353000.csv')]
syncLabeledVideos(vidFileList, labelFileList, os.path.join(dlcFolder, 'projects', 'SeedEating-Sarah-2019-08-26', 'videos'), '20190806-x4-day02-crackedseed-video-03-synced')

d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
d:\anaconda\envs\cv36\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Frame Rate: 30.0
[INFO] cleaning up...


In [159]:
writer.release()

NameError: name 'writer' is not defined

In [103]:
bodyParts['nose']['x'][0]

'393.8775372505188'